In [21]:
import numpy as np
import neat
import importlib
import environment 
importlib.reload(environment)

<module 'environment' from '/Users/lorenzoleuzzi/Documents/GitHub/lifelong_evolutionary_swarms/environment.py'>

In [22]:
initial_setting = {
    'agents': np.array([[5, 5], [10, 10], [15, 15]], dtype=float),
    'blocks': np.array([[4, 16], [13, 5], [16, 4]], dtype=float),
    'colors': np.array([environment.RED, environment.RED, environment.BLUE], dtype=int)
}
env = environment.Environment(objective = [(environment.RED, environment.TOP_EDGE), (environment.BLUE, environment.RIGHT_EDGE)],
                   size = environment.SIMULATION_ARENA_SIZE, 
                   n_agents = 3, 
                   n_blocks = 3,
                   n_neighbors = 3,
                   sensor_range = environment.SIMULATION_SENSOR_RANGE,
                   sensor_angle = 360,
                   max_distance_covered_per_step = environment.SIMULATION_MAX_DISTANCE,
                   sensitivity = 0.5,
                   initial_setting = initial_setting)

In [16]:
def calculate_fitnesses(genomes, config, verbose=False):
    
    for genome_id, genome in genomes:
        
        genome.fitness = 0.0
        obs, _ = env.reset()
        
        net = neat.nn.FeedForwardNetwork.create(genome, config)
        obs, _ = env.reset()

        for _ in range(50):
            
            # Extract data for all agents at once for vectorized computation
            types = obs["neighbors"][:, :, 0]
            distances = obs["neighbors"][:, :, 1] / env.max_distance_covered_per_step
            directions = obs["neighbors"][:, :, 2] / env.sensor_angle
            carrying_block = obs["carrying"]

            # Vectorized one-hot encoding for types
            types_one_hot_encoded = np.eye(env.n_types)[types.astype(int)]

            # Adjust carrying_block indices for one-hot encoding
            carrying_block[carrying_block == -1] = 0  # Map -1 to 0 for one-hot encoding
            carrying_block_one_hot_encoded = np.eye(env.n_types-2)[carrying_block - 2]

            # Reshape distances and directions for concatenation
            distances = distances.reshape(env.n_agents, -1, 1)
            directions = directions.reshape(env.n_agents, -1, 1)

            # Concatenate all inputs
            nn_inputs = np.concatenate([types_one_hot_encoded, distances, directions], axis=2)
            nn_inputs = nn_inputs.reshape(env.n_agents, -1)

            # Concatenate carrying_block_one_hot_encoded
            nn_inputs = np.concatenate([nn_inputs, carrying_block_one_hot_encoded], axis=1)

            nn_outputs = []
            for nn_input in nn_inputs:
                nn_output = net.activate(nn_input)
                nn_outputs.append(nn_output)
            
            nn_outputs = np.array(nn_outputs)
            # Apply softmax to the first 3 outputs
            exp_outputs = np.exp(nn_outputs[:, :3])
            softmax_outputs = exp_outputs / np.sum(exp_outputs, axis=1, keepdims=True)

            # Apply sigmoid to the last 2 outputs
            sigmoid_outputs = 1 / (1 + np.exp(-nn_outputs[:, 3:]))

            # Combine outputs
            nn_outputs = np.concatenate([softmax_outputs, sigmoid_outputs], axis=1)

            # Generate actions for all agents
            actions = [{
                "action": np.argmax(nn_output[:2]),
                "move": [
                    nn_output[3] * env.max_distance_covered_per_step,
                    nn_output[4] * env.sensor_range
                ]
            } for nn_output in nn_outputs]

            obs, reward, done, _ = env.step(actions)
            genome.fitness += reward

            if verbose:
                print(actions)
                env.print_env()
                print(reward)
            
            if done:
                break

In [17]:
# Set configuration file
config_path = "./neat_config.txt"
config = neat.config.Config(neat.DefaultGenome, neat.DefaultReproduction,
                            neat.DefaultSpeciesSet, neat.DefaultStagnation, config_path)

# Create core evolution algorithm class
p = neat.Population(config)

# Add reporter for fancy statistical result
p.add_reporter(neat.StdOutReporter(True))
stats = neat.StatisticsReporter()
p.add_reporter(stats)

# Run NEAT
winner = p.run(calculate_fitnesses, 10)


 ****** Running generation 0 ****** 

Population's average fitness: -150.00000 stdev: 0.00000
Best fitness: -150.00000 - size: (5, 220) - species 1 - id 1
Average adjusted fitness: 0.000
Mean genetic distance 1.211, standard deviation 0.243
Population of 150 members in 1 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1    0   150   -150.0    0.000     0
Total extinctions: 0
Generation time: 5.512 sec

 ****** Running generation 1 ****** 

Population's average fitness: -150.00000 stdev: 0.00000
Best fitness: -150.00000 - size: (5, 220) - species 1 - id 1
Average adjusted fitness: 0.000
Mean genetic distance 1.252, standard deviation 0.253
Population of 150 members in 1 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1    1   150   -150.0    0.000     1
Total extinctions: 0
Generation time: 4.423 sec (4.968 average)

 ****** Running generation 2 ****** 

Population's average fitness: -15

In [18]:
calculate_fitnesses([(1, winner)], config, verbose=True)

[{'action': 1, 'move': [2.8085592930059047, 2.182785153329692]}, {'action': 1, 'move': [2.8085592930059047, 2.182785153329692]}, {'action': 1, 'move': [2.8085592930059047, 2.182785153329692]}]
. . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . O . . . .
. . . . . . 0 . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . 1 . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . .
. . . .